In [1]:
from kafkaproducer import Kafka_Producer
import pandas as pd
import threading
import time
import random

In [ ]:
stop_flag = False #Need the producer to produce the records till stop_flag = False, for better control over the production

In [3]:
def get_env_params():
    return pd.read_csv('../env_params/env_params.csv',index_col=0, header=None).squeeze().to_dict()

In [4]:
def runmain(producer, num_orders,params):
    if num_orders == 0:
         while not stop_flag:
            num_orders = random.randint(1,10)
            producer.generateRecords(num_orders)
            producer.connect_as_producer([params['bootstrapservers']])

            #Sleep used to give time between sending each record
            time.sleep(random.randint(100,200))
            producer.send_records_orders()
            producer.send_records_items()

    else:
        producer.generateRecords(num_orders)
        producer.connect_as_producer([params['bootstrapservers']])
        producer.send_records_orders()
        producer.send_records_items()     

In [5]:
if __name__ == "__main__":
    
    countries = ["India","USA","UK"]
    num_orders = 0
    # Input parameters
    choice = input("Do you want it to run on its own or manually? (Enter 'M' for Manual or 'A' for Automation): ")
    if(choice.upper() == 'M'):
        countries = input("Enter the country to generate orders for (e.g., USA, UK, India): ").split(',')
        num_orders = int(input("Enter the number of fake orders to generate: "))

    params = get_env_params()

    #Create object of kafka producer to start producing
    kafkaobjects = []
    for country in countries:
        kafkaobjects.append(Kafka_Producer(country))
    
    
    # Create threads for each producer function
    threads = []
    for obj in kafkaobjects:
        thread = threading.Thread(target=runmain, args=(obj,num_orders,params))
        threads.append(thread)
        thread.start()

    try:
        while not stop_flag:
            time.sleep(0.1)  # Main thread does something here, could be anything
    except KeyboardInterrupt:
        stop_flag = True
        print("\nKeyboard interruption detected. Stopping all threads...")

    finally:
        # Wait for all threads to complete
        for thread in threads:
            thread.join()
        print("All threads have been stopped.")
    


Keyboard interruption detected. Stopping all threads...
